In [ ]:
# Use Spark SQL with DataFrame, find the amount spend  by country code
from pyspark.sql.types import DoubleType,StructType, StringType, IntegerType, DateType
schema = StructType() \
         .add("InvoiceNo", StringType(), True) \
         .add("StockCode", StringType(), True) \
         .add("Description", StringType(), True) \
         .add("Quantity", IntegerType(), True) \
         .add("InvoiceDate", DateType(), True) \
         .add("UnitPrice", DoubleType(), True) \
         .add("CustomerID", IntegerType(), True) \
         .add("Country", StringType(), True)


df = spark.read.format("csv") \
                .option("header", True) \
                .schema(schema) \
                .option("dateFormat", "MM/dd/yyyy HH:mm")\
                .load("/FileStore/tables/data.csv")

df.show(2)


+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity|InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6| 2010-12-01| 2.55| 17850|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6| 2010-12-01| 3.39| 17850|United Kingdom|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
only showing top 2 rows

In [ ]:
from pyspark.sql.functions import col
 
filteredDf = df.filter ("Quantity  > 0 AND Quantity is NOT NULL" )
filteredDf.cache()
filteredDf.show(2)

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity|InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6| 2010-12-01| 2.55| 17850|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6| 2010-12-01| 3.39| 17850|United Kingdom|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
only showing top 2 rows

In [ ]:
projectionDf = filteredDf.select ("Quantity", "UnitPrice",  "Country")
projectionDf.show(2)

+--------+---------+--------------+
Quantity|UnitPrice| Country|
+--------+---------+--------------+
 6| 2.55|United Kingdom|
 6| 3.39|United Kingdom|
+--------+---------+--------------+
only showing top 2 rows

In [ ]:
from pyspark.sql.functions import split, month, year
from pyspark.sql.functions import asc, desc, avg, col, count, avg, sum
import pyspark.sql.functions as F
# 1/17/2011 17:44 MONTH/DAY/YEAR
# projectionDf.withColumn("Date", split_col.getItem(0))\
 

from pyspark.sql.functions import concat, lit
projectionDf2 = projectionDf\
                            .withColumn("Amount",  col("Quantity") * col("UnitPrice"))\
                            .drop("Quantity", "UnitPrice")
                            
                            

#split_date = split(projectionDf["Date"],"/") 

projectionDf2.show(20)

+--------------+------------------+
 Country| Amount|
+--------------+------------------+
United Kingdom|15.299999999999999|
United Kingdom| 20.34|
United Kingdom| 22.0|
United Kingdom| 20.34|
United Kingdom| 20.34|
United Kingdom| 15.3|
United Kingdom| 25.5|
United Kingdom|11.100000000000001|
United Kingdom|11.100000000000001|
United Kingdom| 54.08|
United Kingdom|12.600000000000001|
United Kingdom|12.600000000000001|
United Kingdom| 30.0|
United Kingdom| 9.899999999999999|
United Kingdom| 25.5|
United Kingdom|14.850000000000001|
United Kingdom| 19.9|
United Kingdom| 17.85|
United Kingdom| 17.85|
United Kingdom| 31.8|
+--------------+------------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import desc

resultDf = projectionDf2.groupBy("Country")\
                    .agg(sum("Amount"))\
                    .withColumnRenamed("sum(Amount)", "Amount")\
                    .sort(desc("Amount"))

resultDf.show(20)

+---------------+------------------+
 Country| Amount|
+---------------+------------------+
 United Kingdom| 9003097.963999517|
 Netherlands|285446.33999999997|
 EIRE| 283453.9599999997|
 Germany| 228867.1400000001|
 France|209715.11000000002|
 Australia| 138521.31|
 Spain|61577.110000000015|
 Switzerland| 57089.89999999996|
 Belgium| 41196.34000000001|
 Sweden| 38378.33|
 Japan| 37416.37|
 Norway| 36165.44|
 Portugal|33747.100000000006|
 Finland|22546.079999999998|
 Singapore| 21279.29|
Channel Islands|20450.440000000006|
 Denmark| 18955.34|
 Italy|17483.239999999998|
 Hong Kong|15691.799999999994|
 Cyprus|13590.379999999997|
+---------------+------------------+
only showing top 20 rows

In [ ]:
outputFile = "/FileStore/tables/assessment-5/results.csv"
# repartition will suffle data, good to be used between your analytics work, not end of the file writing
# coalesce - will not suffle data, reduce  the partition, good to be used before producing file
resultDf = resultDf.coalesce(1)
resultDf.write.mode("overwrite").option("header", True).format("csv").save(outputFile)